# Creating a convolutional network

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers

conv_network = models.Sequential()
conv_network.add(layers.Conv2D(20, (5,5), activation='relu', input_shape=(232, 154, 3)))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.Conv2D(20, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.MaxPooling2D(3,3))
conv_network.add(layers.Conv2D(20, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.Conv2D(10, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.MaxPooling2D(3,3))
conv_network.add(layers.Flatten())
conv_network.add(layers.Dense(4, activation='softmax'))

optimizer=tf.keras.optimizers.Adam(learning_rate=0.02)

conv_network.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

conv_network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 228, 150, 20)      1520      
                                                                 
 dropout (Dropout)           (None, 228, 150, 20)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 146, 20)      10020     
                                                                 
 dropout_1 (Dropout)         (None, 224, 146, 20)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 48, 20)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 70, 44, 20)        10020     
                                                        

# Loading in the data

In [20]:
data_dir = "/Users/kerickwalker/src/dis/deep_learning/bat_data"

img_width = 154
img_height = 232
batch_size = 128

# Load in the training data
training_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Load in validation data
validation_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2008 files belonging to 4 classes.
Using 1607 files for training.
Found 2008 files belonging to 4 classes.
Using 401 files for validation.


# Training convolutional network

In [ ]:
conv_network.fit(training_data, validation_data=validation_data, epochs=10)

# Transfer Learning with MobileNetV2

#### Convert dataset to numpy array for preprocessing

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

In [24]:
img_size = (232, 154)  # MobileNetV2 input size
batch_size = 32
data_dir = "/Users/kerickwalker/src/dis/deep_learning/bat_data"

In [25]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 2008 images belonging to 4 classes.


In [26]:
base_model = MobileNetV2(
    input_shape=(232, 154, 3),
    include_top=False,
    weights='imagenet'
)

In [27]:
for layer in base_model.layers:
    layer.trainable = False

In [28]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [29]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)

Epoch 1/10


2023-11-30 18:29:04.053048: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


62/62 [==============================] - 38s 560ms/step - loss: 0.6074 - accuracy: 0.7657
Epoch 2/10
62/62 [==============================] - 44s 715ms/step - loss: 0.2596 - accuracy: 0.9018
Epoch 3/10
62/62 [==============================] - 50s 809ms/step - loss: 0.2202 - accuracy: 0.9165
Epoch 4/10
62/62 [==============================] - 52s 833ms/step - loss: 0.1985 - accuracy: 0.9276
Epoch 5/10
62/62 [==============================] - 51s 822ms/step - loss: 0.1963 - accuracy: 0.9276
Epoch 6/10
62/62 [==============================] - 57s 922ms/step - loss: 0.2040 - accuracy: 0.9236
Epoch 7/10
62/62 [==============================] - 57s 912ms/step - loss: 0.1698 - accuracy: 0.9357
Epoch 8/10
62/62 [==============================] - 52s 834ms/step - loss: 0.1672 - accuracy: 0.9332
Epoch 9/10
62/62 [==============================] - 50s 795ms/step - loss: 0.1603 - accuracy: 0.9408
Epoch 10/10
62/62 [==============================] - 48s 778ms/step - loss: 0.1711 - accuracy: 0.9332
